## 0. 구글 드라이브 마운트 및 필수 설치

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/Shareddrives/TLDR/1. ETRI/BertSum"

/content/drive/Shareddrives/TLDR/1. ETRI/BertSum


In [3]:
!pip install -r requirements.txt
!sudo apt-get install libxml-parser-perl

     |████████████████████████████████| 60 kB 3.0 MB/s 
     |████████████████████████████████| 123 kB 11.4 MB/s 
     |████████████████████████████████| 124 kB 41.5 MB/s 
     |████████████████████████████████| 131 kB 44.5 MB/s 
     |████████████████████████████████| 79 kB 6.5 MB/s 
     |████████████████████████████████| 8.3 MB 41.2 MB/s 
     |████████████████████████████████| 138 kB 48.0 MB/s 
     |████████████████████████████████| 127 kB 42.9 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191620 sha256=ae99d5df829becc78e1fc5ade41dcf92b39a55a448d82d9dfabc16414e6035a0
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/ffb9a1f51b2b00fee42e7f67f5a5d8e10c67d048cda09ccd57
Successfully built pyrouge
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

In [10]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path 'pyrouge/tools/ROUGE-1.5.5'

  Using cached https://github.com/bheinzerling/pyrouge/archive/master.zip
Name: pyrouge
Version: 0.1.3
Summary: A Python wrapper for the ROUGE summarization evaluation package.
Home-page: https://github.com/noutenki/pyrouge
Author: Benjamin Heinzerling, Anders Johannsen
Author-email: benjamin.heinzerling@h-its.org
License: LICENSE.txt
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 
Cloning into 'pyrouge'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393
Receiving objects: 100% (393/393), 298.74 KiB | 2.06 MiB/s, done.
Resolving deltas: 100% (109/109), done.
Checking out files: 100% (355/355), done.
2021-11-29 23:25:29,350 [MainThread  ] [INFO ]  Set ROUGE home directory to pyrouge/tools/ROUGE-1.5.5.


In [11]:
!chmod 755 -R ./pyrouge

In [12]:
%cd ./pyrouge/tools/ROUGE-1.5.5/data
!mv WordNet-2.0.exc.db WordNet-2.0.exc.db.orig
!perl WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db

/content/drive/Shareddrives/TLDR/1. ETRI/BertSum/src/pyrouge/tools/ROUGE-1.5.5/data


In [13]:
%cd "../../../../"

/content/drive/Shareddrives/TLDR/1. ETRI/BertSum/src


## 1. 모델 Train
- learning rate 2e-3
- dropout rate 0.1
- batch size 2000
- training step 30000
- warmup step 6000
---
### 1. Simple Classifier 인코더

In [ ]:
!python train.py -mode train -encoder classifier -dropout 0.1 -bert_data_path ../bert_data/korean -model_path ../models/ -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 2000 -decay_method noam -train_steps 30000 -accum_count 1 -log_file ../logs/training_log.txt -use_interval true -warmup_steps 6000 -bert_model ../content/ETRIBert -bert_config_path ../content/ETRIBert/bert_config.json -temp_dir ./temp

[2021-11-28 16:29:17,708 INFO] Device ID 0
[2021-11-28 16:29:17,709 INFO] Device cuda
[2021-11-28 16:29:17,749 INFO] loading archive file ../content/ETRIBert
[2021-11-28 16:29:17,751 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}

[2021-11-28 16:29:22,935 INFO] Summarizer(
  (bert): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30349, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        

### 2. Transformer 인코더
- ff_size - 2048
- internal_layers - 2
- heads - 8

In [ ]:
!python train.py -mode train -encoder transformer -dropout 0.1 -bert_data_path ../bert_data/korean -model_path ../models/ -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 2000 -decay_method noam -train_steps 30000 -accum_count 1 -log_file ../logs/training_transformer_log.txt -use_interval true -warmup_steps 6000 -bert_model ../content/ETRIBert -bert_config_path ../content/ETRIBert/bert_config.json -temp_dir ./temp -ff_size 2048 -inter_layers 2 -heads 8

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[2021-11-26 12:16:10,151 INFO] Loading train dataset from ../bert_data/korean.train.8268.bert.pt, number of examples: 5
[2021-11-26 12:16:11,941 INFO] Loading train dataset from ../bert_data/korean.train.14397.bert.pt, number of examples: 5
[2021-11-26 12:16:13,828 INFO] Loading train dataset from ../bert_data/korean.train.10524.bert.pt, number of examples: 5
[2021-11-26 12:16:15,602 INFO] Loading train dataset from ../bert_data/korean.train.4295.bert.pt, number of examples: 5
[2021-11-26 12:16:17,522 INFO] Loading train dataset from ../bert_data/korean.train.605.bert.pt, number of examples: 5
[2021-11-26 12:16:19,257 INFO] Loading train dataset from ../bert_data/korean.train.12652.bert.pt, number of examples: 5
[2021-11-26 12:16:21,150 INFO] Loading train dataset from ../bert_data/korean.train.758.bert.pt, number of examples: 5
[2021-11-26 12:16:22,472 INFO] Loading train dataset from ../bert_data/korean.train.1100.bert.pt, number of examples: 5
[20

### 3. RNN 인코더

In [ ]:
!python train.py -mode train -encoder rnn -dropout 0.1 -bert_data_path ../bert_data/korean -model_path ../models/ -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 2000 -decay_method noam -train_steps 30000 -accum_count 1 -log_file ../logs/training_rnn_log.txt -use_interval true -warmup_steps 6000 -rnn_size 768 -bert_model ../content/ETRIBert -bert_config_path ../content/ETRIBert/bert_config.json -temp_dir ./temp

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[2021-11-26 18:56:14,570 INFO] Loading train dataset from ../bert_data/korean.train.1564.bert.pt, number of examples: 5
[2021-11-26 18:56:16,234 INFO] Loading train dataset from ../bert_data/korean.train.10037.bert.pt, number of examples: 5
[2021-11-26 18:56:17,930 INFO] Loading train dataset from ../bert_data/korean.train.2892.bert.pt, number of examples: 5
[2021-11-26 18:56:19,722 INFO] Loading train dataset from ../bert_data/korean.train.7209.bert.pt, number of examples: 5
[2021-11-26 18:56:21,230 INFO] Loading train dataset from ../bert_data/korean.train.6356.bert.pt, number of examples: 5
[2021-11-26 18:56:22,850 INFO] Loading train dataset from ../bert_data/korean.train.2972.bert.pt, number of examples: 5
[2021-11-26 18:56:24,636 INFO] Loading train dataset from ../bert_data/korean.train.6330.bert.pt, number of examples: 5
[2021-11-26 18:56:26,298 INFO] Loading train dataset from ../bert_data/korean.train.13582.bert.pt, number of examples: 5
[2

### 4. Test Score
- RNN, Transformers, Classifier의 ROGUE Scores
- 1) RNN (배치 사이즈 1000)

In [15]:
!python train.py -encoder rnn -mode test -bert_data_path ../bert_test_data/korean -bert_model ../content/ETRIBert -model_path ../models/ -test_from ../models/trained_RNN/model_step_15000.pt -visible_gpus 0 -gpu_ranks 0 -batch_size 2000 -log_file ../logs/log_rnn_15000_test.txt -result_path ../results/ -test_all -block_trigram true -bert_config_path ../content/ETRIBert/bert_config.json -temp_dir ./temp/

[2021-11-30 00:01:39,399 INFO] Loading checkpoint from ../models/trained_RNN/model_step_15000.pt
Namespace(accum_count=1, batch_size=2000, bert_config_path='../content/ETRIBert/bert_config.json', bert_data_path='../bert_test_data/korean', bert_model='../content/ETRIBert', bert_model_type='etri', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='rnn', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='../logs/log_rnn_15000_test.txt', lr=1, max_grad_norm=0, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/', rnn_size=768, save_checkpoint_steps=5, seed=666, temp_dir='./temp/', test_all=True, test_from='../models/trained_RNN/model_step_15000.pt', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_steps=8000, world_size=1)
[2021-11-30 00:01:59,279 INFO] loading archive file .

In [ ]:
!chmod 755 -R ./pyrouge

#### RNN 15000 steps ROGUE Score

In [ ]:
!pyrouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl -e pyrouge/tools/ROUGE-1.5.5/data -c 95 -m -r 1000 -n 2 -a ./temp/tmpj94caips/rouge_conf.xml

---------------------------------------------
1 ROUGE-1 Average_R: 0.47817 (95%-conf.int. 0.46940 - 0.48676)
1 ROUGE-1 Average_P: 0.32052 (95%-conf.int. 0.31385 - 0.32724)
1 ROUGE-1 Average_F: 0.34396 (95%-conf.int. 0.33759 - 0.35047)
---------------------------------------------
1 ROUGE-2 Average_R: 0.19386 (95%-conf.int. 0.18675 - 0.20095)
1 ROUGE-2 Average_P: 0.12789 (95%-conf.int. 0.12266 - 0.13342)
1 ROUGE-2 Average_F: 0.13819 (95%-conf.int. 0.13285 - 0.14369)
---------------------------------------------
1 ROUGE-L Average_R: 0.46396 (95%-conf.int. 0.45540 - 0.47212)
1 ROUGE-L Average_P: 0.31016 (95%-conf.int. 0.30367 - 0.31665)
1 ROUGE-L Average_F: 0.33268 (95%-conf.int. 0.32635 - 0.33880)


- 2) Transformers (배치 2000)

In [14]:
!python train.py -mode test -encoder transformer -bert_data_path ../bert_test_data/korean -bert_model ../content/ETRIBert -model_path ../models/ -test_from ../models/trained_transformer/model_step_15000.pt -visible_gpus 0 -gpu_ranks 0 -batch_size 2000 -accum_count 1 -log_file ../logs/log_transformer_15000_test.txt -test_all -block_trigram true -bert_config_path ../content/ETRIBert/bert_config.json -temp_dir ./temp

[2021-11-29 23:26:12,728 INFO] Loading checkpoint from ../models/trained_transformer/model_step_15000.pt
Namespace(accum_count=1, batch_size=2000, bert_config_path='../content/ETRIBert/bert_config.json', bert_data_path='../bert_test_data/korean', bert_model='../content/ETRIBert', bert_model_type='etri', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='transformer', ff_size=2048, gpu_ranks=[0], heads=8, hidden_size=128, inter_layers=2, log_file='../logs/log_transformer_15000_test.txt', lr=1, max_grad_norm=0, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/cnndm', rnn_size=512, save_checkpoint_steps=5, seed=666, temp_dir='./temp', test_all=True, test_from='../models/trained_transformer/model_step_15000.pt', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_steps=8000, world_size=1)
[2021-11-29 23:2

In [ ]:
!python train.py -mode test -bert_data_path ../bert_test_data/korean -bert_model ../content/ETRIBert -model_path ../models/ -test_from ../models/trained_transformer/model_step_25000.pt -visible_gpus 0 -gpu_ranks 0 -batch_size 2000 -accum_count 1 -log_file ../logs/log_transformer_25000_test.txt -test_all -block_trigram true -bert_config_path ../content/ETRIBert/bert_config.json -temp_dir ./temp

[2021-11-28 15:45:47,674 INFO] Loading checkpoint from ../models/trained_transformer/model_step_25000.pt
Namespace(accum_count=1, batch_size=2000, bert_config_path='../content/ETRIBert/bert_config.json', bert_data_path='../bert_test_data/korean', bert_model='../content/ETRIBert', bert_model_type='etri', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='transformer', ff_size=2048, gpu_ranks=[0], heads=8, hidden_size=128, inter_layers=2, log_file='../logs/log_transformer_25000_test.txt', lr=1, max_grad_norm=0, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/cnndm', rnn_size=512, save_checkpoint_steps=5, seed=666, temp_dir='./temp', test_all=True, test_from='../models/trained_transformer/model_step_25000.pt', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_steps=8000, world_size=1)
[2021-11-28 15:4

- 3) Classifier 성능

In [ ]:
!chmod 755 -R ./pyrouge
!python train.py -mode test -bert_data_path ../bert_test_data/korean -bert_model ../content/ETRIBert -model_path ../models/ -test_from ../models/trained_classifier/model_step_14000.pt -visible_gpus 0 -gpu_ranks 0 -batch_size 2000 -accum_count 1 -log_file ../logs/log_classifier_15000_test.txt -test_all -block_trigram true -bert_config_path ../content/ETRIBert/bert_config.json -temp_dir ./temp

[2021-11-28 15:02:02,549 INFO] Loading checkpoint from ../models/trained_classifier/model_step_14000.pt
Namespace(accum_count=1, batch_size=2000, bert_config_path='../content/ETRIBert/bert_config.json', bert_data_path='../bert_test_data/korean', bert_model='../content/ETRIBert', bert_model_type='etri', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='classifier', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='../logs/log_classifier_15000_test.txt', lr=1, max_grad_norm=0, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/cnndm', rnn_size=512, save_checkpoint_steps=5, seed=666, temp_dir='./temp', test_all=True, test_from='../models/trained_classifier/model_step_14000.pt', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_steps=8000, world_size=1)
[2021-11-28 15:02:19,

In [ ]:
!pyrouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl -e pyrouge/tools/ROUGE-1.5.5/data -c 95 -m -r 1000 -n 2 -a ./temp/tmpfk43txo1/rouge_conf.xml

Couldn't open ./temp/tmpfk43txo1/rouge_conf.xml:
No such file or directory at pyrouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl line 501.


In [ ]:
%cd src

/content/drive/Shareddrives/TLDR/ETRI_MODEL/BertSum/src


In [ ]:
with open('../results/_step5000.gold','r') as f:
    gold = f.readlines()

In [ ]:
gold[3]

"위/NNG 종문회/NNG 가/JKS 부동산실/NNG 권리자명/NNG 의/JKG 등기/NNG 에/JKB 관하/VV ㄴ/ETM 법률/NNG 시행/NNG 전/NNG 에/JKB 피고/NNG (/SS 문중원/NNG 부락민/NNG )/SS 를/JKO 상대/NNG 로/JKB 명의/NNG 신탁/NNG 해지/NNG 를/JKO 원인/NNG 으로/JKB 하/VV ㄴ/ETM 소유권지분/NNG 이/VCP 기/ETN 소송/NNG 을/JKO 제기/NNG 하/XSV 였/EP 다가/EC 법원/NNG 으로부터/JKB 부동산/NNG 의/JKG 소유/NNG 권/XSN 이/JKS 원고/NNG 부/NNP 락/NNG 에게/JKB 있/VV 다는/ETM 이유/NNG 로/JKB 패소/NNG 판결/NNG 을/JKO 선고/NNG 받/VV 자/EC 판결/NNG 확정일/NNG 로부터/JKB 1/SN 년/NNB 이내/NNG 에/JKB 원고/NNG 부락/NNG 이/JKS 피고/NNG 를/JKO 상대/NNG 로/JKB 명의/NNG 신탁/NNG 해지/NNG 를/JKO 원인/NNG 으로/JKB 하/VV ㄴ/ETM 소유/NNG 권/XSN 이/JKS 전등기청구/NNG 의/JKG 소/NNG 를/JKO 제기/NNG 하/XSV ㄴ/ETM 경우/NNG ,/SP<q>위/NNG 와/JKB 같/VA 은/ETM 일련/NNG 의/JKG 소송/NNG 들/XSN 은/JX 전체/NNG 가/JKS 일체/NNG 가/JKC 되/VV 어/EC 같/VA 은/ETM 법/NNG 제/XPN 11/SN 조/NNB 제/XPN 4/SN 항/NNG 소정/NNG 의/JKG '/SS 부동산물/NNG 권/XSN 에/JKB 관하/VV ㄴ/ETM 쟁송/NNG '/SS 에/JKB 해당/NNG 되/XSV ㄴ다고/EC 봄/NNG 이/JKS 상당/NNG 하/XSA 므로/EC ,/SP<q>위/NNG 일련/NNG 의/JKG 소송/NNG 의/JKG 계속/NNG 중/NNB 에/JKB 는/JX 기존/NNG 의/JKG 명의/NNG 신탁/NNG 관계/NNG 가/JKS 실효/NNG 되/XSV 지

In [ ]:
with open('../results/_step5000.candidate','r') as f:
    candidate = f.readlines()

In [ ]:
candidate[3]

"위/NNG 일련/NNG 의/JKG 소송/NNG 의/JKG 계속/NNG 중/NNB 에/JKB 는/JX 기존/NNG 의/JKG 명의/NNG 신탁/NNG 관계/NNG 가/JKS 실효/NNG 되/XSV 지/EC 않/VX 는다고/EC 보/VV ㄴ/ETM 사례/NNG ./SF<q>[/SS 1/SN ]/SS 부동산실/NNG 권리자명/NNG 의/JKG 등기/NNG 에/JKB 관하/VV ㄴ/ETM 법률/NNG 제/XPN 11/SN 조/NNB 제/XPN 4/SN 항/NNG 소정/NNG 의/JKG '/SS 부동산/NNG 물권/NNG 에/JKB 관하/VV ㄴ/ETM 쟁송/NNG '/SS 이/VCP 라/EC 하/VV ㅁ/ETN 은/JX 명의/NNG 신탁자/NNG 가/JKS 당자자/NNG 로서/JKB 해당/NNG 부동산/NNG 에/JKB 관하/VV 여/EC 자신/NNG 이/JKS 실권리자/NNG 이/VCP ㅁ/ETN 을/JKO 주장/NNG 하/XSV 여/EC 이/NP 를/JKO 공/NNG 적/XSN 으로/JKB 확인/NNG 받/VV 기/ETN 위하/VV ㄴ/ETM 쟁송/NNG 이/VCP 면/EC 족하/VA 다/EF ./SF<q>자연/NNG 부락/NNG 의/JKG 경우/NNG 도/JX 부락/NNG 이/JKS 보유/NNG 하/XSV ㄴ/ETM 부동산/NNG 을/JKO 부락민/NNG 들/XSN 에게/JKB 명의/NNG 신탁/NNG 하/XSV 여/EC 놓/VX 는/ETM 관행/NNG 이/JKS 있/VV 어/EC 오/VX 았/EP 고/EC 또한/MAG 같/VA 은/ETM 법/NNG 이/JKS 방지/NNG 하/XSV 려고/EC 하/VX 는/ETM 부동산등기제도/NNG 의/JKG 악용/NNG 소지/NNG 도/JX 크/VA 지/EC 않/VX 다고/EC 하/VV ㄹ/ETM 것/NNB 이/VCP 어서/EC\n"